In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
from selenium.webdriver.chrome.service import Service

In [2]:
# 웹 드라이버 생성 함수 구현
def create_driver() :
    service=Service()
    options=webdriver.ChromeOptions()
    driver=webdriver.Chrome(service=service, options=options)
    return driver

In [3]:
driver=create_driver()

In [4]:
# 보통 동적으로 브라우저 핸들링 할때 약간의 지연이 있을 수 있으므로 기라리는것이 일반적
driver.get('https://www.naver.com')
driver.implicitly_wait(2)

In [5]:
# 네이버사이트는 창 크기에 따라 다른 표현을 진행함:소스코드도 달라짐
# 창크기는 고정해놓고 크롤링 하는것이 좋음
# max 크기로 브라우저 크기 설정
driver.maximize_window()

In [7]:
# 쇼핑메뉴 객체를 지칭하는 css selector
sel = '#shortcutArea > ul > li:nth-child(4) > a'

In [8]:
# 쇼핑메뉴 클릭
driver.find_element(By.CSS_SELECTOR,sel).click()

#### 클릭을 통해서 쇼핑페이지로 들어가면 브라우저 탭(윈도우)이 하나 더 열림
#### driver가 핸들링 할 수 있는 창이 2개가 됨
#### 기본 창은 첫번째 접속했던 네이버사이트 창임
- 현재 드라이버의 활성창을 쇼핑으로 변경해야 함

In [9]:
print(driver.window_handles)
# 결과 리스트의 0번 index가 제일 먼저 열린 탭
# 쇼핑은 1번 index

['7700B1B659131B86070B528B8CA5766D', 'D1F483DDD05FFD3CF635DE38A2675233']


In [10]:
# 현재 driver의 활성 탭을 변경
driver.switch_to.window(driver.window_handles[1])

In [11]:
sel = '#gnb-gnb > div._gnb_header_area_150KE > div > div._gnbLogo_gnb_logo_3eIAf > div > div._gnbSearch_gnb_search_3O1L2 > form > div._gnbSearch_inner_2Zksb > div > input'
sel2='._gnbSearch_inner_2Zksb > div > input'

In [14]:
search=driver.find_element(By.CSS_SELECTOR,sel2)
search.send_keys('아이폰13')

In [15]:
# enter키 전송을 하면 클릭과 같은 효과가 나타남
# from selenium.webdriver.common.keys import Keys : Keys 클래스는 키보드 키값을 상수로 저장하고 있음
# Keys.ENTER : 키보드의 엔터키값
search.send_keys(Keys.ENTER)

#### 스크롤을 최대로 진행해서 현재 페이지에서 디스플레이 되는 모든 제품을 전부 표현
- 현재 페이지에 대해서 전송된 모든 제품에 대하여 동일 소스코드로 생성하기 위함
- 스크롤과 관련된 자바스크립트 코드 사용
- 스크립트 객체 window : 브라우저 탭의 프레임을 포함
    - window.scrollY : 브라우저탭의 yscroll의 현재 위치를 저장하고 있는 java script 속성

In [18]:
# 탭안의 스크롤바의 현재 위치를 저장
before_h = driver.execute_script('return window.scrollY')

In [19]:
# 무한 스크롤 : 키보드의 End키는 스크롤을 진행함
# END키는 현재 창의 가장 끝으로 내려감
# 숨겨진 창까지 모두 내려가도록 무한스크롤 진행
while True : 
    # 창의 가장 아래로 스크롤 내림, 스크롤이 창 바닦에 닿으면 새로운 창이 확장됨
    driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
    # 스크롤 사이 페이지 로딩시간 대기
    time.sleep(1)
    # 스크롤 후 높이
    after_h = driver.execute_script('return window.scrollY')
    print(after_h)
    if after_h==before_h:
        break
    before_h = after_h

1612
3764
5908
7854
9697
9866
9866


In [20]:
sel = '#content > div.style_content__xWg5l > div.basicList_list_basis__uNBZx > div > div'
sel2 = 'div.basicList_list_basis__uNBZx > div > div'
items=driver.find_elements(By.CSS_SELECTOR,sel2)
print(len(items))

46


In [21]:
items[17].get_attribute('innerHTML')

'<div class="product_item__MDtDF"><div class="product_inner__gr8QR"><div class="product_img_area__cUrko"><div class="thumbnail_thumb_wrap__RbcYO _wrapper"><a target="_blank" rel="noopener" class="thumbnail_thumb__Bxb6Z linkAnchor" data-nclick="N=a:lst*C.image,r:15,i:29029192586" data-i="29029192586" data-ms="0" href="https://cr.shopping.naver.com/adcr.nhn?x=uIZpY%2BHte3Ix6BIf3o0Vj%2F%2F%2F%2Fw%3D%3DsT9Lq6xfD6nUSuzLvmevpzEm1kV9eo5X6zTn%2FFGstfrJN4NgkrPIjlczCBjhk2dSMY9bOhLtB3mMizrQOC0Pm56dcvfdl93VC71t8WUudX5WDfUP3whulDFunP657F3PQsQxI0r%2BbPmFsVWLl0g7V3Mv%2BjB2erv%2BZ%2Brrs5VHOiTQgGE7%2BZatKjf5hEf1XctkKwzZoEE1irv0jVFjUiTpIZJbkCnZt3PKDnGtnPfTpWXV9gLRMuJH34gK%2BdKxxm2wLfHX%2FKvLeMRhy2z4sWQA%2F16sJF1TG8DXvuOXRWh2aiSE1QmeK3ZPITzKHcbEnR%2BI8mXpE%2FWYm5Ai0tHt21rYEpkBFUYfhI7uH5Fgl4yiCwH3Ye9CsbWrnsboog%2B94RL%2FI0Xaie43S1ci44hGLx1E05YPk2vryFU100lCcPyTJO%2BBFx37WIyrziE4Ps6WrDu%2FET9Lvfhn9NmZebq06Rt58vUK94HazsIJk%2FjsIwcFBS2bPU6pntGCurhu%2FV8uonTYh5X6MB3JdbnBohOi8QY2mskVuBY5V8KvlWSxQpSde83U81Gi44tU

In [24]:
# 제품명 추출
# #content > div.style_content__xWg5l > div.basicList_list_basis__uNBZx > div > div:nth-child(17) > div > div > div.product_info_area__xxCTi > div.product_title__Mmw2K
sel='div.product_title__Mmw2K'
items[17].find_element(By.CSS_SELECTOR,sel).text

'아이폰 13 프로 맥스 512GB [자급제]'

In [25]:
# 가격 추출
items[17].find_element(By.CSS_SELECTOR,'span.price em').text

'1,932,000'

In [26]:
# 제품의 실제 판매처 url
sel='div.product_title__Mmw2K > a'
items[17].find_element(By.CSS_SELECTOR,sel).get_attribute('href')

'https://cr.shopping.naver.com/adcr.nhn?x=uIZpY%2BHte3Ix6BIf3o0Vj%2F%2F%2F%2Fw%3D%3DsT9Lq6xfD6nUSuzLvmevpzEm1kV9eo5X6zTn%2FFGstfrJN4NgkrPIjlczCBjhk2dSMY9bOhLtB3mMizrQOC0Pm56dcvfdl93VC71t8WUudX5WDfUP3whulDFunP657F3PQsQxI0r%2BbPmFsVWLl0g7V3Mv%2BjB2erv%2BZ%2Brrs5VHOiTQgGE7%2BZatKjf5hEf1XctkKwzZoEE1irv0jVFjUiTpIZJbkCnZt3PKDnGtnPfTpWXV9gLRMuJH34gK%2BdKxxm2wLfHX%2FKvLeMRhy2z4sWQA%2F16sJF1TG8DXvuOXRWh2aiSE1QmeK3ZPITzKHcbEnR%2BI8mXpE%2FWYm5Ai0tHt21rYEpkBFUYfhI7uH5Fgl4yiCwH3Ye9CsbWrnsboog%2B94RL%2FI0Xaie43S1ci44hGLx1E05YPk2vryFU100lCcPyTJO%2BBFx37WIyrziE4Ps6WrDu%2FET9Lvfhn9NmZebq06Rt58vUK94HazsIJk%2FjsIwcFBS2bPU6pntGCurhu%2FV8uonTYh5X6MB3JdbnBohOi8QY2mskVuBY5V8KvlWSxQpSde83U81Gi44tUFH1uRUKbnASPQezdPirNIEhU3YcDKwV2g5K6QNQ9yt4scyrUny9Y4h26y5o2wVXHl6Nqb%2FoiWMPZUQdAiJdwXirBRdnaxfvrBbw%3D%3D&nvMid=29029192586&catId=50001519'

In [29]:
for item in items :
    try : 
        name = item.find_element(By.CSS_SELECTOR,'div.product_title__Mmw2K').text
        price = item.find_element(By.CSS_SELECTOR,'span.price em').text
        link = item.find_element(By.CSS_SELECTOR,'div.product_title__Mmw2K > a').get_attribute('href')
    except :
        name='판매중단'
        price='판매중단'
        link='판매중단'
    print(name,price,link)

판매중단 판매중단 판매중단
판매중단 판매중단 판매중단
판매중단 판매중단 판매중단
아이폰 13 256GB [자급제] 1,090,000 https://cr.shopping.naver.com/adcr.nhn?x=RxNcWADSciOgtV%2FsGi8qYv%2F%2F%2Fw%3D%3Dsc3U7SiJHOyi4d2G4%2Fe%2Fit7pQ1Q4D7%2BVlwz0a5mnNpLyGgjNtjvRyY8TaEm3AQ5UwpsniIgIYBYrlvx%2BGvIHMBkqo6n7qF3YPofV6Aa07MyYiIUwXTeY6KaMdFuJ%2FYxPVJ9ydSTe1GkC4ZQID6E0KIN9%2BKy%2Fm0d%2BgYWHtEf61Jo5CNclKwvLyA%2Fys%2F%2FHPPajLymj3I0Ayg2aYacGIsBoPOmNmcbgLr0lu2cCLGgyX7c9vsepmwRk8o%2FCB4DgHhnjVnI5GUgxMf4ORiqZn3C1LBauKWQN4b7WstTXVRp4V%2Fd2Kx7G3krbJiQXzryuwwOqJS6XYwqGx4vHGBp6my58hJLqadzp7FSPm7GuQ5%2FecuMJKjRp2D9pkBhcJdqPQCXkTJTyOtdpNmqs6E%2FnW3NDtTlNCrJpfkPGyKA93w5qrtIQLSv5SYzis5SVdiBAX6a8R%2FCOsDULMzhyS3tcNpyUq6M%2FFIf4yGPCeFlgQd%2BW1h9K7A%2F0vSPti8cHY4o2fastw5nGX4TfzRlG2ahGMx%2BrSF%2F%2BKxsxur5vNCX%2FLomt4deMRpOFzCbaM6OU97uoPifjrcD0oBzKELcJajvyNgOqD7GHbocQTLQf9T5R84DzsDVCFbDf4gHI5rSMJU1BWcIOg&nvMid=29029192591&catId=50001519
아이폰 13 미니 128GB [자급제] 689,000 https://cr.shopping.naver.com/adcr.nhn?x=imbWd4IkmeY%2FMrJZPlUttv%2F%2F%2Fw%3D%3DsFWjQGb7DyAte

아이폰 13 프로 128GB [SKT] 1,183,300 https://cr.shopping.naver.com/adcr.nhn?x=Dn9xjiN72whMdnwS7eTYD%2F%2F%2F%2Fw%3D%3Ds37bx7t9GTnHyCxVhBsdQcQ%2Bc1pMeHmWGEnaryb009PHrlQPyQuHoHFsppYbPEdHaFH4nlWBCNH%2BwoOgbM%2B15x%2BjldOx9yz1%2FKxgIAWAWipgcftp%2FN9DUiqZqkHD4BNgut3RRWGCmZoS%2FKQL60CE8toZnlrSZr8PDPdBCvs8uWtdeJxHcKH4rYUhc5SxvGs%2FQ96DW%2BGycTBZMVki5iCCJNyJO7jbxAC%2FhHJnJbE9pD225SET%2FmTh%2Bwr3iKnDGr9FR6WMcVzhunHGcajqoxfsnFduvhaZJBMtBPpV%2BXYtitfLBz7%2BORHa%2BX777%2F5U82WuFRDpykpOqWZgZqWcBmVwRDU7JgJbZICKNejrN5R%2BKAjM6JIeyjRJJG8AQSjlBThBKI4LX2ExoCUK9EE2u%2ByICs0WEh0pNn3CKdGSnP1%2Bc0KciNvEAyH7XT6ObGe95mRdUsw%2FEBFoDxdMAZ4HSpGP%2F1iPyGZXVC4%2BpXxewNqrW%2B3wYe1OqyckzwcdzigJZeXyj8GeP5QCF6mm4B5aN9MhUjBSTRf%2FzNcc6lvtvuIDPb3idSlPOZBN6V54TTL6u%2FzFsRGAH42RQxV7IsHw4AovglV9z4Dnyyu74Bk5nJq6vipnke%2FVP%2BgM94BPZkP5YJ%2Bex&nvMid=29030177588&catId=50000246
Apple 아이폰 13 프로 256GB [LG U+] 1,299,200 https://cr.shopping.naver.com/adcr.nhn?x=chpc1nywpQhnn4Prt2uODv%2F%2F%2Fw%3D%3Dsy6%2BaJ1kA6ThSdOP3FP7%2B%2F1FdrQ8kJM

애플 애플 아이폰13 프로 (13Pro) 256GB 자급제폰 기기변경 KT 직영점 388,000 https://cr.shopping.naver.com/adcr.nhn?x=PGFQcC5EG6WC2k%2FOZsRDFf%2F%2F%2Fw%3D%3Ds%2BKJexHn06TEvU8DwchCC9GIfPByxZkUXZeWoS6P3Jt2GgjNtjvRyY8TaEm3AQ5UwSeSINsdQkhMpCDgM848kaUqo6n7qF3YPofV6Aa07MyYiIUwXTeY6KaMdFuJ%2FYxPVywodAotKjUjRzELemDJMasNHgaylo8oOLD%2BoSlOgwQ0CHQNNLjv%2FUbY4TLZErh461tlHnLGeH66LUwoQyPsAcCicmLKJk6k%2Be6tx6IIlLcicgh%2BsF9kq5kDuiyg4Ws17ueCfFDuywxl3T1PQ6PKMjF5qtMG%2FJ0PUtQjclakIAMgNcCV1xS50IcUo9%2Bq5Y6MQ%2B7jB1VhXzkM0R6YdA6grP%2FXB4m7nCt%2B9FQWAi7AA2KiGHVDOkn7f1QQ95OyVLZv0tAyLH4krdKJGz3Ch%2BOvRWJK%2Fvswp1de1u13FUfBXK7L0X42k%2FBckeZhywkyvb1fDD7kZLB013vW0e14edtQzSet3NUsjkTVb2bXVZECmd%2BVW1PGJRaj7cap0I9XfbYUpXZ76KJZClTaLoisbxYrSEMFROLqvoDSpBs1RWpvH0%2BA81Gi44tUFH1uRUKbnASPQhgL9sQPcYLlZAiSaaH3d7p9XPiLstL23DiVPlV7ExBDG1V%2FGGXE%2B9wKVhJyN3%2FK5DhLBwu%2FFQS5xWYf5w6G3jQ%3D%3D&nvMid=39558601637&catId=50001519
아이폰13 Mini 128G 블랙 미개봉 새제품 725,000 https://cr.shopping.naver.com/adcr.nhn?x=lEqH2JI2UjjKP0pBNhNi4f%2F%2F%2Fw%3D%3Ds1%2Bv6G

In [55]:
### 현재 페이지의 제품정보 df 생성
name=[]
info=[]
link=[]

In [56]:
# 상품정보 전체 추출
sel2 = 'div.basicList_list_basis__uNBZx > div > div'
items=driver.find_elements(By.CSS_SELECTOR,sel2)

for item in items :
    try : 
        name_tmp=item.find_element(By.CSS_SELECTOR,'div.product_title__Mmw2K').text
        info_tmp=item.find_element(By.CSS_SELECTOR,'span.price em').text
        link_tmp=item.find_element(By.CSS_SELECTOR,'div.product_title__Mmw2K > a').get_attribute('href')

    except :
        continue
    name.append(name_tmp)
    info.append(info_tmp)
    link.append(link_tmp)

In [58]:
len(name),len(info),len(link)

(37, 37, 37)

In [60]:
df_fin = pd.DataFrame({'품명':name, '가격':info, 'link':link})

In [61]:
df_fin.to_csv('crawl_data/네이버쇼핑_아이폰13.csv')

- 두개 탭이 열려 있어서 close 두번 진행

In [ ]:
driver.close()

In [64]:
driver.switch_to.window(driver.window_handles[0])

In [65]:
driver.close()